Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [0]:
%env CUDA_VISIBLE_DEVICES=""

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [0]:
url = 'http://mattmahoney.net/dc/'
data_root = './data'
def maybe_download(fname, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  dst_fname = os.path.join(data_root, fname)
  if not os.path.exists(dst_fname):
    dst_fname, _ = urlretrieve(url + fname, dst_fname)
  statinfo = os.stat(dst_fname)
  if statinfo.st_size == expected_bytes:
    print('Found/verified %s: Stored it in %s' % (fname, dst_fname))
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + fname + '. Can you get to it with a browser?')
  return dst_fname

filename = maybe_download('text8.zip', 31344016)

In [0]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

In [0]:
print(text[:100])
print([''] * 10)


Create a small validation set.

In [0]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

Utility functions to map characters to vocabulary IDs and back.

In [0]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print("'%c', '%c', '%c'"%(id2char(1), id2char(26), id2char(0)))

Function to generate a training batch for the LSTM model.

In [0]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)
print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

In [0]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  # Cross Entropy: avoid "nan" faults by lower bounding prob to 1e-10
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (row [[a, b, c]]) of prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random row of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [0]:
num_nodes = 64

graph = tf.Graph()
with tf.device('/cpu:0'), graph.as_default():
  
  # Parameters:
  # Input gate: input (W: Wx), previous output (U: Uh), and bias (b).
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # Above LSTM cell involves 4 matrix multiplications with the input, and
  # 4 matrix multiplications with the output.
  # Using a single matrix multiply for each, and variables that are 4 times larger
  def lstm_cell(e, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    eo = tf.concat(1, [e, o])
    ip_fo_op_up_xm = tf.concat(0,
        [tf.concat(1, [ix, fx, cx, ox]), tf.concat(1, [im, fm, cm, om])])
    ip_fo_op_up = tf.matmul(eo, ip_fo_op_up_xm) + tf.concat(1, [ib, fb, cb, ob])
    ip_fo_op, update = tf.split_v(value=ip_fo_op_up, size_splits=[3*num_nodes, num_nodes], split_dim=1)
    input_gate, forget_gate, output_gate = tf.split(split_dim=1, num_split=3, value=tf.sigmoid(ip_fo_op))
    state = forget_gate * state + input_gate * tf.tanh(update)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(0, train_labels), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [0]:
num_steps = 7001
summary_frequency = 100

with tf.device('/cpu:0'), tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Validation set perplexity: 4.46
Rewrote 4 matrix multiplications for input/output to 1 matrix multiplication for input/output. 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [0]:
class BatchIdGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=[self._batch_size], dtype=np.int32)
    for b in range(self._batch_size):
      batch[b] = char2id(self._text[self._cursor[b]])
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def random_bi_distribution():
  """ Generate a random row of bi-char probabilities."""
  b = np.random.uniform(0.0,1.0,size=[1,vocabulary_size*vocabulary_size])
  return b/np.sum(b, 1)[:,None]

def bi_char_batches(batches):
  """ Generate a corresponding batch of bigram characters from
  a batch of unigram characters """
  num_batches = len(batches)
  assert num_batches > 0
  batch_prev = np.zeros(batches[0].shape, dtype=np.int32)
  bi_batches =[]
  for idx, batch in enumerate(batches):
    b = batch_prev*vocabulary_size + batch
    bi_batches.append(b)
    batch_prev = batch
  return bi_batches

def ids2BiId(id1, id2):
  return id1*vocabulary_size + id2

def biId2Ids(bi_id):
  return bi_id//vocabulary_size, bi_id%vocabulary_size

def biId2bichar(bi_id):
  id1, id2 = biId2Ids(bi_id)
  return id2char(id1) + id2char(id2)

def bichar2BiId(ch1, ch2):
  return ids2BiId(char2id(ch1), char2id(ch2))

def bi_chars(batch):
  assert batch.ndim == 1
  res = [biId2bichar(id) for id in batch]
  return res

def bi_batches2string(bi_batches):
  """Convert a sequence of bi_batches back into their (most likely) string
  representation."""
  s = [''] * len(bi_batches[0])
  for b in bi_batches:
    one_char = [c[1:] for c in bi_chars(b)]
    s = [''.join(x) for x in zip(s, one_char)]
  return s

train_idbatches = BatchIdGenerator(train_text, batch_size, num_unrollings)
valid_idbatches = BatchIdGenerator(valid_text, 1, 1)
print(bi_batches2string(bi_char_batches(train_idbatches.next())))
print(bi_batches2string(bi_char_batches(train_idbatches.next())))
print(bi_batches2string(bi_char_batches(valid_idbatches.next())))
print(bi_batches2string(bi_char_batches(valid_idbatches.next())))
print("ids2BiId: ch1={}, ch2={}, biId={}".format('a', 'n', bichar2BiId('a', 'n')))
assert biId2bichar(bichar2BiId('a','n')) == 'an'
assert bichar2BiId(biId2bichar(41)[0], biId2bichar(41)[1]) == 41

In [0]:
def logprob_sparse(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  # Cross Entropy: avoid "nan" faults by lower bounding prob to 1e-10
  predictions[predictions < 1e-10] = 1e-10
  assert predictions.ndim == 2 and labels.ndim == 1
  return np.mean(-np.log(predictions)[np.arange(labels.shape[0]), labels])

tmp = np.arange(12).reshape(3,4)
predictions = tmp/np.sqrt(np.sum(tmp*tmp, axis=1, keepdims=True))
loss = logprob_sparse(predictions, np.array([1,2,0]))
print("Loss=%.5f"%loss)
assert np.abs(loss - 0.93926) < 1e-5

In [0]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with tf.device('/cpu:0'), graph.as_default():  
  # Parameters:
  # Embedding Layer:
  # Two Characters are mapped into an embedding layer
  # uniform_init = tf.random_uniform_initializer(-1.0, 1.0)
  uniform_init = tf.random_uniform_initializer(-0.1, 0.1)
  with tf.variable_scope("Embedding", initializer=uniform_init) as scope:
    embeds = tf.get_variable("Embeds",
                             [vocabulary_size*vocabulary_size, embedding_size], initializer=uniform_init)
  
  # Input gate: input (W: Wx), previous output (U: Uh), and bias (b).
  ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))

  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))

  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))

  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))

  # Variables saving state/output (i.e. ct/ht in paper) across unrollings:
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # Above LSTM cell involves 4 matrix multiplications with the input, and
  # 4 matrix multiplications with the output.
  # Using a single matrix multiply for each, and variables that are 4 times larger
  def lstm_cell(e, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    eo = tf.concat(1, [e, o])
    ip_fo_op_up_xm = tf.concat(0,
                            [tf.concat(1, [ix, fx, cx, ox]), tf.concat(1, [im, fm, cm, om])])
    ip_fo_op_up = tf.matmul(eo, ip_fo_op_up_xm) + tf.concat(1, [ib, fb, cb, ob])
    ip_fo_op, update = tf.split_v(value=ip_fo_op_up, size_splits=[3*num_nodes, num_nodes], split_dim=1)
    input_gate, forget_gate, output_gate = tf.split(split_dim=1, num_split=3, value=tf.sigmoid(ip_fo_op))
    state = forget_gate * state + input_gate * tf.tanh(update)
    return output_gate * tf.tanh(state), state

  # INPUT DATA
  # 1. Input/Labels are provided via a list of contiguous characters
  # of length=(num_unrollings + 1)
  train_data_ph = list()
  for _ in range(num_unrollings + 1):
    train_data_ph.append(tf.placeholder(tf.int32, shape=[batch_size], name="train_data"))
  # 2. Dropout: Only applied on input/projection
  # (i.e. x/y in paper and not in internal state/output ct/ht)
  dropout_ph = tf.placeholder(tf.float32, name= "dropout")

  # inputs are bicharId extracted from train_data
  train_inputs = list()
  prev_data = tf.zeros([batch_size], dtype=tf.int32)
  for i in range(num_unrollings):
    cur_data = train_data_ph[i]
    # bicharId = id1*vocabulary_size + id2
    train_inputs.append(vocabulary_size*prev_data + cur_data)
    prev_data = cur_data

  # labels are inputs shifted by one time step.
  train_labels = train_data_ph[1:]  

  # Unroll LSTM loop upto num_unrollings times.
  outputs = list()
  output = saved_output
  state    = saved_state
  for i in train_inputs:
    # apply dropout at input/embedding layer or output layer:
    # i.e. when layers are crossed - not in the recurrent layer
    train_embed = tf.nn.dropout(tf.nn.embedding_lookup(embeds, i), dropout_ph)
    output, state = lstm_cell(train_embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # apply dropout at input/embedding layer or output layer:
    # i.e. when layers are crossed - not in the recurrent layer
    # Classifier
    projection = tf.nn.dropout(tf.concat(0, outputs), dropout_ph)
    logits = tf.nn.xw_plus_b(projection, w, b)
    labels = tf.concat(0, train_labels)
    xent = tf.nn.sparse_softmax_cross_entropy_with_logits(
      logits=logits, labels=labels)
    loss = tf.reduce_mean(xent)

  # Optimizer.
  global_step = tf.Variable(0)
  #  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.75, staircase=True)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1 of biCharId, no unrolling, and no dropout
  sample_input_ph = tf.placeholder(tf.int32, shape=[1], name="sample_input")
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))

  # create a group tensor op to reset sample output/state
  reset_sample_state_op = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))

  sample_embed = tf.nn.embedding_lookup(embeds, sample_input_ph)
  sample_output, sample_state = lstm_cell(
    sample_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [0]:
num_steps = 7001
summary_frequency = 100

with tf.device('/cpu:0'), tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_idbatches.next()
    feed_dict = { dropout_ph: 0.95 }
    for i in range(num_unrollings + 1):
      feed_dict[train_data_ph[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob_sparse(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          prevCharId = 0
          nextCharId = sample_distribution(random_distribution()[0])
          biId = ids2BiId(prevCharId, nextCharId)
          sentence = id2char(nextCharId)
          reset_sample_state_op.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input_ph: [biId]})
            prevCharId = nextCharId
            nextCharId = sample_distribution(prediction[0])
            biId = ids2BiId(prevCharId, nextCharId)
            sentence += id2char(nextCharId)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state_op.run()
      valid_logprob = 0
      b_prev = np.random.randint(vocabulary_size)
      for _ in range(valid_size):
        b = valid_idbatches.next()
        biId = ids2BiId(b_prev, b[0])
        predictions = sample_prediction.eval({sample_input_ph: biId})
        valid_logprob = valid_logprob + logprob_sparse(predictions, b[1])
        b_prev = b[0]
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Validation set perplexity: 3.98
num_nodes = 64; embedding_size = 128; Dropout 0.95
Learning Rate 10.0, global_step, 5000, 0.1, staircase=True)

Reimplement unigram using embedding as input instead of directly feeding 1-hot vector

In [0]:
num_nodes = 64
embedding_size = num_nodes

graph = tf.Graph()
with tf.device('/cpu:0'), graph.as_default():  
  # Parameters:
  # Embedding Layer:
  # Two Characters are mapped into an embedding layer
  # uniform_init = tf.random_uniform_initializer(-1.0, 1.0)
  uniform_init = tf.random_uniform_initializer(-0.1, 0.1)
  with tf.variable_scope("Embedding", initializer=uniform_init) as scope:
    embeds = tf.get_variable("Embeds",
                             [vocabulary_size, embedding_size], initializer=uniform_init)
  
  # Input gate: input (W: Wx), previous output (U: Uh), and bias (b).
  ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))

  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))

  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))

  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))

  # Variables saving state/output (i.e. ct/ht in paper) across unrollings:
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # Above LSTM cell involves 4 matrix multiplications with the input, and
  # 4 matrix multiplications with the output.
  # Using a single matrix multiply for each, and variables that are 4 times larger
  def lstm_cell(e, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    eo = tf.concat(1, [e, o])
    ip_fo_op_up_xm = tf.concat(0,
                            [tf.concat(1, [ix, fx, cx, ox]), tf.concat(1, [im, fm, cm, om])])
    ip_fo_op_up = tf.matmul(eo, ip_fo_op_up_xm) + tf.concat(1, [ib, fb, cb, ob])
    ip_fo_op, update = tf.split_v(value=ip_fo_op_up, size_splits=[3*num_nodes, num_nodes], split_dim=1)
    input_gate, forget_gate, output_gate = tf.split(split_dim=1, num_split=3, value=tf.sigmoid(ip_fo_op))
    state = forget_gate * state + input_gate * tf.tanh(update)
    return output_gate * tf.tanh(state), state

  # INPUT DATA
  # 1. Input/Labels are provided via a list of contiguous characters
  # of length=(num_unrollings + 1)
  train_data_ph = list()
  for _ in range(num_unrollings + 1):
    train_data_ph.append(tf.placeholder(tf.int32, shape=[batch_size], name="train_data"))
  # 2. Dropout: Only applied on input/projection
  # (i.e. x/y in paper and not in internal state/output ct/ht)
  dropout_ph = tf.placeholder(tf.float32, name= "dropout")

  train_inputs = train_data_ph[:num_unrollings]
  # labels are inputs shifted by one time step.
  train_labels = train_data_ph[1:]  

  # Unroll LSTM loop upto num_unrollings times.
  outputs = list()
  output = saved_output
  state    = saved_state
  for i in train_inputs:
    # apply dropout at input/embedding layer or output layer:
    # i.e. when layers are crossed - not in the recurrent layer
    train_embed = tf.nn.dropout(tf.nn.embedding_lookup(embeds, i), dropout_ph)
    output, state = lstm_cell(train_embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # apply dropout at input/embedding layer or output layer:
    # i.e. when layers are crossed - not in the recurrent layer
    # Classifier
    projection = tf.nn.dropout(tf.concat(0, outputs), dropout_ph)
    logits = tf.nn.xw_plus_b(projection, w, b)
    labels = tf.concat(0, train_labels)
    xent = tf.nn.sparse_softmax_cross_entropy_with_logits(
      logits=logits, labels=labels)
    loss = tf.reduce_mean(xent)

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1 of charId, no unrolling, and no dropout
  sample_input_ph = tf.placeholder(tf.int32, shape=[1], name="sample_input")
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))

  # create a group tensor op to reset sample output/state
  reset_sample_state_op = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))

  sample_embed = tf.nn.embedding_lookup(embeds, sample_input_ph)
  sample_output, sample_state = lstm_cell(
    sample_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [0]:
# num_steps = 100001
# summary_frequency = 1000
num_steps = 7001
summary_frequency = 100

with tf.device('/cpu:0'), tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_idbatches.next()
    feed_dict = { dropout_ph: 0.90 }
    for i in range(num_unrollings + 1):
      feed_dict[train_data_ph[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob_sparse(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          charId = sample_distribution(random_distribution()[0])
          sentence = id2char(charId)
          reset_sample_state_op.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input_ph: [charId]})
            charId = sample_distribution(prediction[0])
            sentence += id2char(charId)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state_op.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_idbatches.next()
        predictions = sample_prediction.eval({sample_input_ph: b[0]})
        valid_logprob = valid_logprob + logprob_sparse(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

In [0]:
Validation set perplexity: 4.73
num_nodes = 64; embedding_size = 128; Dropout 0.95
Learning Rate 10.0, global_step, 5000, 0.1, staircase=True)

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

Test Beam Search Implementation. Source: https://github.com/tensorflow/tensorflow/issues/654

In [0]:
from __future__ import division
import tensorflow as tf

In [0]:
graph = tf.Graph()
with tf.device('/cpu:0'), graph.as_default():  
    beam_size = 3 # Number of hypotheses in beam.
    num_symbols = 5 # Output vocabulary size.
    embedding_size = 10
    num_steps = 3
    embedding = tf.zeros([num_symbols, embedding_size])
    output_projection = None

    # log_beam_probs: list of [beam_size, 1] Tensors
    #  Ordered log probabilities of the `beam_size` best hypotheses
    #  found in each beam step (highest probability first).
    # beam_symbols: list of [beam_size] Tensors 
    #  The ordered `beam_size` words / symbols extracted by the beam
    #  step, which will be appended to their corresponding hypotheses
    #  (corresponding hypotheses found in `beam_path`).
    # beam_path: list of [beam_size] Tensor
    #  The ordered `beam_size` parent indices. Their values range
    #  from [0, `beam_size`), and they denote which previous
    #  hypothesis each word should be appended to.
    log_beam_probs, beam_symbols, beam_path  = [], [], []
    def beam_search(prev, i):
        # Tensor Dimensions
        # Prev: [batch_size x] beam_size x num_symbols
        # log_beam_probs[ind]: [batch_size x] beam_size x 1
        if output_projection is not None:
            prev = tf.nn.xw_plus_b(
                prev, output_projection[0], output_projection[1])

        # Compute 
        #  log P(next_word, hypothesis) = 
        #  log P(next_word | hypothesis)*P(hypothesis) =
        #  log P(next_word | hypothesis) + log P(hypothesis)
        # for each hypothesis separately, then join them together 
        # on the same tensor dimension to form the example's 
        # beam probability distribution:
        # [P(word1, hypothesis1), P(word2, hypothesis1), ...,
        #  P(word1, hypothesis2), P(word2, hypothesis2), ...]

        # If TF had a log_sum_exp operator, then it would be 
        # more numerically stable to use: 
        #   probs = prev - tf.log_sum_exp(prev, reduction_dims=[1])
        probs = tf.log(tf.nn.softmax(prev))
        # i == 1 corresponds to the input being "<GO>", with
        # uniform prior probability and only the empty hypothesis
        # (each row is a separate example).
        if i > 1:
            probs = tf.reshape(probs + log_beam_probs[-1], 
                               [-1, beam_size * num_symbols])

        # Get the top `beam_size` candidates and reshape them such
        # that the number of rows = batch_size * beam_size, which
        # allows us to process each hypothesis independently.
        ## top_k returns 'k' best_probs and corresponding indices for every row
        ## squeeze just swallows all 1 dimensions in a tensor
        ## stop_gradient: Stops back propagation: when executed in a graph,
        ##                             this op outputs its input tensor as-is and 
        best_probs, indices = tf.nn.top_k(probs, beam_size)
        # indices = tf.stop_gradient(tf.squeeze(tf.reshape(indices, [-1, 1])))
        # best_probs = tf.stop_gradient(tf.reshape(best_probs, [-1, 1]))
        indices = tf.stop_gradient(tf.squeeze(tf.reshape(indices, [-1, beam_size, 1])))
        best_probs = tf.stop_gradient(tf.expand_dims(tf.squeeze(best_probs), -1))

        symbols = indices % num_symbols # Which word in vocabulary.
        beam_parent = indices // num_symbols # Which hypothesis it came from.

        beam_symbols.append(symbols)
        beam_path.append(beam_parent)
        log_beam_probs.append(best_probs)
        return tf.nn.embedding_lookup(embedding, symbols)

    # Setting up graph.
    inputs = [tf.placeholder(tf.float32, shape=[None, 1, num_symbols])] + [
      tf.placeholder(tf.float32, shape=[None, beam_size, num_symbols])
      for i in range(num_steps-1)]
    for i in range(num_steps):
        beam_search(inputs[i], i + 1)


In [0]:
with tf.device('/cpu:0'), tf.Session(graph=graph) as session:
    # Running the graph.
    input_vals = [0, 0, 0]
    l = np.log
    eps = -10 # exp(-10) ~= 0
    # These values mimic the distribution of vocabulary words
    # from each hypothesis independently (in log scale since
    # they will be put through exp() in softmax).
    input_vals[0] = np.array([0, eps, l(2), eps, l(3)]).reshape(1,1,5)
    # Step 1 beam hypotheses =
    # (1) Path: [4], prob = log(1 / 2)
    # (2) Path: [2], prob = log(1 / 3)
    # (3) Path: [0], prob = log(1 / 6)

    input_vals[1] = np.array([[l(1.2), 0, 0, l(1.1), 0], # Path [4] 
                              [0,   eps, eps, eps, eps], # Path [2]
                              [0,  0,   0,   0,   0]]) .reshape(1,3,5)  # Path [0]
    # Step 2 beam hypotheses =
    # (1) Path: [2, 0], prob = log(1 / 3) + log(1)
    # (2) Path: [4, 0], prob = log(1 / 2) + log(1.2 / 5.3)
    # (3) Path: [4, 3], prob = log(1 / 2) + log(1.1 / 5.3)

    input_vals[2] = np.array([[0,  l(1.1), 0,   0,   0], # Path [2, 0]
                              [eps, 0,   eps, eps, eps], # Path [4, 0]
                              [eps, eps, eps, eps, 0]]).reshape(1,3,5)  # Path [4, 3]
    # Step 3 beam hypotheses =
    # (1) Path: [4, 0, 1], prob = log(1 / 2) + log(1.2 / 5.3) + log(1)
    # (2) Path: [4, 3, 4], prob = log(1 / 2) + log(1.1 / 5.3) + log(1)
    # (3) Path: [2, 0, 1], prob = log(1 / 3) + log(1) + log(1.1 / 5.1)

    # input_feed = {inputs[i]: input_vals[i][:beam_size, :]
    input_feed = {inputs[i]: input_vals[i][:beam_size, :] for i in xrange(num_steps)} 
    output_feed = beam_symbols + beam_path + log_beam_probs
    outputs = session.run(output_feed, feed_dict=input_feed)


In [0]:
expected_beam_symbols = np.array([[4, 2, 0], [0, 0, 3], [1, 4, 1]])
expected_beam_path = np.array([[0, 0, 0], [1, 0, 0], [1, 2, 0]])

print("predicted beam_symbols vs. expected beam_symbols")
for ind, predicted in enumerate(outputs[:num_steps]):
  print("(", predicted, ",", expected_beam_symbols[ind], ")")

print("\npredicted beam_path vs. expected beam_path")
for ind, predicted in enumerate(outputs[num_steps:num_steps * 2]):
  print("(", predicted, ",", expected_beam_path[ind], ")")

print("\nlog beam probs")
for log_probs in outputs[2 * num_steps:]:
  print(log_probs)

In [0]:
with tf.device('/cpu:0'), tf.Session(graph=graph) as session:
    # Running the graph.
    l = np.log
    eps = -10 # exp(-10) ~= 0
    # These values mimic the distribution of vocabulary words
    # from each hypothesis independently (in log scale since
    # they will be put through exp() in softmax).
    inp_vals = np.array([0, eps, l(2), eps, l(3)])
    input_vals[0] = np.vstack((inp_vals, inp_vals)).reshape(2,1,5)
    # Step 1 beam hypotheses =
    # (1) Path: [4], prob = log(1 / 2)
    # (2) Path: [2], prob = log(1 / 3)
    # (3) Path: [0], prob = log(1 / 6)

    inp_vals = np.array([[l(1.2), 0, 0, l(1.1), 0], # Path [4] 
                              [0,   eps, eps, eps, eps], # Path [2]
                              [0,  0,   0,   0,   0]])   # Path [0]
    input_vals[1] = np.vstack((inp_vals, inp_vals)).reshape(2,3,5)
    # Step 2 beam hypotheses =
    # (1) Path: [2, 0], prob = log(1 / 3) + log(1)
    # (2) Path: [4, 0], prob = log(1 / 2) + log(1.2 / 5.3)
    # (3) Path: [4, 3], prob = log(1 / 2) + log(1.1 / 5.3)

    inp_vals = np.array([[0,  l(1.1), 0,   0,   0], # Path [2, 0]
                         [eps, 0,   eps, eps, eps], # Path [4, 0]
                         [eps, eps, eps, eps, 0]])  # Path [4, 3]
    input_vals[2] = np.vstack((inp_vals,inp_vals)).reshape(2,3,5)
    # Step 3 beam hypotheses =
    # (1) Path: [4, 0, 1], prob = log(1 / 2) + log(1.2 / 5.3) + log(1)
    # (2) Path: [4, 3, 4], prob = log(1 / 2) + log(1.1 / 5.3) + log(1)
    # (3) Path: [2, 0, 1], prob = log(1 / 3) + log(1) + log(1.1 / 5.1)

    input_feed = {inputs[i]: input_vals[i] for i in xrange(num_steps)} 
    output_feed = beam_symbols + beam_path + log_beam_probs
    outputs = session.run(output_feed, feed_dict=input_feed)

In [0]:
expected_beam_symbols = np.array([[[4, 2, 0],[4, 2, 0],], [[0, 0, 3],[0, 0, 3],], [[1, 4, 1],[1, 4, 1]]])
expected_beam_path = np.array([[[0, 0, 0],[0, 0, 0],], [[1, 0, 0],[1, 0, 0],], [[1, 2, 0],[1, 2, 0]]])

print("predicted beam_symbols vs. expected beam_symbols")
for ind, predicted in enumerate(outputs[:num_steps]):
  print("(", predicted, ",", expected_beam_symbols[ind], ")")

print("\npredicted beam_path vs. expected beam_path")
for ind, predicted in enumerate(outputs[num_steps:num_steps * 2]):
  print("(", predicted, ",", expected_beam_path[ind], ")")

print("\nlog beam probs")
for log_probs in outputs[2 * num_steps:]:
  print(log_probs)


https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/
Magic Commands
> %lsmagic
> %env: sets environment variables
> %run: other python file or ipython NB
> %time %timeit: measure slow code
> %prun: measures how much time program spent in each function
> %pdb: automatically turns on python debugging
> !cmd: executes shell commands
Latex Commands directly inserted in Markdown cell
$$ P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)} $$
>%%python3: executes cell in python3 kernel
>%%bash: executes cell in bash kernel
RISE allows create a powerpoint style presentation from existing notebook
$> pip install RISE
$> jupyter-nbextension install rise --py --sys-prefix
$> jupyter-nbextension enable rise --py --sys-prefix